This is to handle migration.

First we can start to create an adjacency matrix

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy.typing as npt
from typing import *
import matplotlib.pyplot as plt

In [3]:
gdf = gpd.read_file("C:/Users/Conor/Desktop/tpsa_hackathon_CONOR/MAIN_FILE.geojson")
gdf = gdf.to_crs({'init': 'EPSG:29902'})
gdf['area_sq_km'] = gdf['geometry'].area / 1_000_000
gdf.info()

C:\Users\Conor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3409 entries, 0 to 3408
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   GUID_            3409 non-null   object  
 1   2022_Population  3409 non-null   float64 
 2   2016_Population  3409 non-null   int32   
 3   2011_Population  3409 non-null   int32   
 4   2006_Population  3371 non-null   float64 
 5   2002_Population  3367 non-null   float64 
 6   1996_Population  3364 non-null   float64 
 7   geometry         3409 non-null   geometry
 8   area_sq_km       3409 non-null   float64 
dtypes: float64(5), geometry(1), int32(2), object(1)
memory usage: 213.2+ KB


The problem with adding neighbours is that it doesn't account for the fact that physical distance from one ED to another is not limited to the fact that two EDs are touching or not. Also, it doesn't account for the fact that EDs across the country can have a lot to do with each other (e.g. Galway and Dublin).

We therefore want to be able to add neighbours that are not just immediately touhcing off of each other.

In [ ]:
# add NEIGHBORS column
gdf["NEIGHBORS"] = None  

for index, electoral_div in gdf.iterrows():   

    # get 'not disjoint' countries
    neighbors = gdf[~gdf.geometry.disjoint(electoral_div.geometry)].index.tolist()

    # remove own name of the country from the list
    neighbors = [ ed for ed in neighbors if electoral_div.index.tolist() != ed ]

    # add names of neighbors as NEIGHBORS value
    gdf.at[index, "NEIGHBORS"] = neighbors

In [ ]:
# Making the adjacency matrix 

# We define an adj matrix list to record the index of neighbors for each ED 
proto_adj_matrix: list = []  # We do this to get around the irregular dimensions of the object
for index in range(len(gdf)): # Run through the EDs
    proto_adj_matrix.append(gdf.iloc[index].NEIGHBORS) # Add the ED's neighbors into proto adj matrix

# Define the Adj matrix as an n x n matrix of zeroes (where n = number of EDs)
adj_matrix: npt.ArrayLike = np.zeros([len(gdf), len(gdf)])

# Cycle through each ED and its respective list of neighbours
# Since an adj matrix is symmetric - this can be improved but numpy is sufficiently fast that at this scale it doesn't matter
for i, ed in enumerate(proto_adj_matrix):
    for neighbor in ed:
        if i != neighbor:
            adj_matrix[i][neighbor] = 1 # Set the relevant positions to 1 to indicate adjacency
        else:
            adj_matrix[i][neighbor] = 0

In [ ]:
# Making the degree matrix

degree_matrix: npt.ArrayLike = np.diag([len(gdf.iloc[index].NEIGHBORS) for index, rows in gdf.iterrows()])